In [114]:
import pickle,regex

In [21]:
import numpy

Reading in Tweepy data, and turning into a dictionary

In [12]:
with open("search_output-2016-04-16.bin") as tweet_file:
    results = []
    while not tweet_file.closed:
        try: 
            results += [pickle.load(tweet_file)]
        
        except EOFError: tweet_file.close()

In [151]:
exclusion_list = ["http"]
table = string.maketrans("","")

In [186]:
word_dict = {}
for r in results[:]:
    temp_words = r.split(" ")
    words = []
    for tw in temp_words: words += tw.split("\n") 
    for w in words:
        temp_word = regex.sub(ur"\p{P}+", "", w.lower())
        if not any([e in temp_word for e in exclusion_list ]) or temp_word is not u'':
            if temp_word not in word_dict: word_dict[temp_word] = 0
            word_dict[temp_word] += 1

Sorting and reducing the word list.

Reducing in this context means removing any words that are substrings of other words, and removing the unique entry and adding the substring word's count to the superstring's entry.

In [187]:
#reducing down the word list
word_list = sorted(word_dict.items(),key=lambda x:x[0])
new_word_list = []
word_stack = 0
for i,w in enumerate(word_list[:-1]):
    if w[0] not in word_list[i+1][0]: 
        new_word_list += [(w[0],w[1]+word_stack)]
        word_stack = 0
    else: 
        word_stack += w[1]
        
#sorting by number of times used
word_list = sorted(new_word_list,key=lambda x:x[1])

In [188]:
word_list[-100:]

[(u'managed', 53),
 (u'needed', 53),
 (u'weekendbreakfast', 53),
 (u'mostly', 54),
 (u'online', 54),
 (u'shouldnt', 54),
 (u'thando', 54),
 (u'werent', 54),
 (u'africanamericans', 55),
 (u'against', 55),
 (u'southafrican', 55),
 (u'videos', 56),
 (u'when', 56),
 (u'wouldnt', 56),
 (u'leadership', 57),
 (u'manifestoites', 57),
 (u'mustafizurs', 57),
 (u'liveaction', 59),
 (u'theironmaidens', 59),
 (u'sabcnewslive', 60),
 (u'takeaways', 60),
 (u'launched', 61),
 (u'soaking', 63),
 (u'trumps', 63),
 (u'viable', 63),
 (u'oneil', 64),
 (u'backers', 66),
 (u'getafe', 66),
 (u'theres', 67),
 (u'doable', 68),
 (u'if', 68),
 (u'liked', 68),
 (u'whod', 68),
 (u'justicemalala', 69),
 (u'japanearthquake', 70),
 (u'popefrancis', 70),
 (u'theyd', 74),
 (u'herebastards', 75),
 (u'todays', 75),
 (u'allah', 76),
 (u'headache', 77),
 (u'timed', 77),
 (u'years', 77),
 (u'upcoming', 78),
 (u'myanc', 80),
 (u'nowhere', 82),
 (u'canadas', 83),
 (u'overalls', 85),
 (u'wasangamayhem', 89),
 (u'nobody', 90),
 